In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2016, 2024))

In [3]:
DIR = os.getcwd() + "/nfl_predictor"
DATA_DIR = os.path.join(DIR, "data")
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")


In [4]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i) #prevents us from scrapping too fast
        
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout Error on {url}")
            continue
        else:
            break
        
    return html

In [8]:
async def scrape_season(season):
    url = f"https://www.pro-football-reference.com/years/{season}/games.htm"
    path = f"{season}_games.htm"
    save_path = os.path.join(STANDINGS_DIR, path)
    if os.path.exists(save_path):
        return
    
    html = await get_html(url, "#all_games")
    
    print(save_path)
    with open(save_path, 'w+') as f:
        f.write(html)
    
    

In [9]:
for season in SEASONS:
    await scrape_season(season)

2016 NFL Regular Season Schedule | Pro-Football-Reference.com
/Users/brandonhart/Documents/nfl_predictor/data/standings/2016_games.htm
2017 NFL Regular Season Schedule | Pro-Football-Reference.com
/Users/brandonhart/Documents/nfl_predictor/data/standings/2017_games.htm
2018 NFL Regular Season Schedule | Pro-Football-Reference.com
/Users/brandonhart/Documents/nfl_predictor/data/standings/2018_games.htm
2019 NFL Regular Season Schedule | Pro-Football-Reference.com
/Users/brandonhart/Documents/nfl_predictor/data/standings/2019_games.htm
2020 NFL Regular Season Schedule | Pro-Football-Reference.com
/Users/brandonhart/Documents/nfl_predictor/data/standings/2020_games.htm
2021 NFL Regular Season Schedule | Pro-Football-Reference.com
/Users/brandonhart/Documents/nfl_predictor/data/standings/2021_games.htm
2022 NFL Regular Season Schedule | Pro-Football-Reference.com
/Users/brandonhart/Documents/nfl_predictor/data/standings/2022_games.htm
2023 NFL Regular Season Schedule | Pro-Football-Referen

In [26]:
standings_files = os.listdir(STANDINGS_DIR)

In [27]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".htm" in l]
    box_scores = [f"https://www.pro-football-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, 'w+') as f:
            f.write(html)
        

In [ ]:

    for f in standings_files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

In [29]:
standings_files

['2021_games.htm',
 '2019_games.htm',
 '2020_games.htm',
 '2018_games.htm',
 '2016_games.htm',
 '2023_games.htm',
 '2017_games.htm',
 '2022_games.htm']